# Model Evaluation of Performance on Individual Outcomes
This notebook compares and evaluates the performance of previously fitted models in predicting individual outcomes. Unlike the previous model evaluations which used the entire sample, these evaluations use reduced datasets comprising of only indidivudals with the outcome of interest, and a control group with no suicide-related outcomes at all. These reduced datasets circumvent problems with calculating metrics such as specificity and precision for individual outcomes in the full data.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
from matplotlib import pyplot 
from numpy import mean
from numpy import std
from sklearn.calibration import calibration_curve
from sklearn.metrics import confusion_matrix as confusion
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score as ppv
from sklearn.metrics import recall_score as recall
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
import joblib
import numpy as np
import pandas as pd
InteractiveShell.ast_node_interactivity = "all"

Suicidal ideation model evaluation

In [7]:
#Recreating full test dataset
Xi_hold=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\Xi_hold.csv')
y_hold=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\y_hold.csv')
Xi_hold.shape
y_hold.shape
#Getting rid of unnamed column- this often occurs after csv is read with the index from excel
Xi_hold= Xi_hold.drop('Unnamed: 0', axis=1)
Xi_hold.to_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\Xi_hold.csv')
testdata=Xi_hold.join(y_hold)
testdata

(729, 1012)

(729, 4)

,fd20a1,fd20c4,pe05c,pe06c2,fd24c1,f13ip1,f17ip1,f17zip1,f17aip1,f17bip1,...,y9test,y9gram,y9num,y9read,y9spel,y9write,si,nssi,att,sitbs
0,-9.0,-9.0,-9.0,-9.0,-9.0,1.0,2.0,2.0,0.0,0.0,...,0.277500,0.742737,0.542185,0.482281,0.769311,0.371608,0.0,0,0.0,0
1,-9.0,-9.0,-9.0,-9.0,-9.0,1.0,2.0,2.0,0.0,0.0,...,0.275579,0.600097,0.520104,0.453224,0.392045,0.371608,1.0,0,0.0,1
2,-9.0,-9.0,-9.0,-9.0,-9.0,1.0,2.0,2.0,0.0,0.0,...,0.277500,0.541766,0.995751,1.102658,0.628022,0.657920,0.0,0,0.0,0
3,-9.0,-9.0,-9.0,-9.0,-9.0,1.0,2.0,2.0,0.0,0.0,...,0.277500,0.109922,-0.049383,-0.033489,-0.384795,-0.459493,0.0,0,0.0,0
4,-9.0,-9.0,-9.0,-9.0,-9.0,1.0,2.0,2.0,0.0,0.0,...,0.277500,0.742737,0.698127,0.716194,0.507059,1.106774,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,-9.0,-9.0,-9.0,-9.0,-9.0,2.0,1.0,1.0,0.0,0.0,...,0.275579,0.146195,-2.510878,-1.214192,0.138221,0.310965,0.0,0,0.0,0
725,-9.0,-9.0,-9.0,-9.0,-9.0,1.0,2.0,2.0,0.0,0.0,...,0.277500,-0.080756,-0.000623,0.031890,0.345940,-0.035989,1.0,0,0.0,1
726,-9.0,-9.0,-9.0,-9.0,-9.0,1.0,2.0,2.0,0.0,0.0,...,0.277500,0.742737,0.942850,0.999020,0.628022,1.033705,0.0,0,0.0,0
727,-9.0,-9.0,-9.0,-9.0,-9.0,1.0,2.0,2.0,0.0,0.0,...,0.279420,0.651075,0.530685,0.419808,0.420799,0.371608,0.0,0,0.0,0


In [44]:
#Examining frequencies of the different combinations of outcomes
testdata[['si', 'nssi', 'att']].value_counts().reset_index(name='count')
#Creating the different subsets of data for evaluating performance on individual outcomes
si_test=testdata[(testdata['si']==1) | ((testdata['att']==0) & (testdata['nssi']==0))]
nssi_test=testdata[(testdata['nssi']==1) | ((testdata['att']==0) & (testdata['si']==0))]
att_test=testdata[(testdata['att']==1) | ((testdata['nssi']==0) & (testdata['si']==0))]

#Checking to see if the datasets were created properly
sre=['si', 'att', 'nssi']
for i in sre:
    print(f'{i}_test frequencies by outcome')
    #Retrieves variable value and concatenates 'i' and '_test'. locals() function is used to construct variable names dynamically
    locals()[f'{i}_test'][['si', 'nssi', 'att']].value_counts().reset_index(name='count')

#Splitting the reduced datasets back into x and y 
#y_holds_si and so on are created just for these evaluations, as opposed to y_hold_si, which contains the outcome of si for the full sample 
#s standing for small, indicating a reduced dataset
for i in sre:
    locals()[f'Xi_holds_{i}']=locals()[f'{i}_test'].drop(columns=['si', 'att', 'nssi','sitbs'])
    locals()[f'y_holds_{i}']=locals()[f'{i}_test'][f'{i}']

,si,nssi,att,count
0,0.0,0,0.0,608
1,1.0,0,0.0,48
2,1.0,1,0.0,23
3,0.0,1,0.0,18
4,1.0,1,1.0,14
5,1.0,0,1.0,11
6,0.0,0,1.0,6
7,0.0,1,1.0,1


si_test frequencies by outcome


,si,nssi,att,count
0,0.0,0,0.0,608
1,1.0,0,0.0,48
2,1.0,1,0.0,23
3,1.0,1,1.0,14
4,1.0,0,1.0,11


att_test frequencies by outcome


,si,nssi,att,count
0,0.0,0,0.0,608
1,1.0,1,1.0,14
2,1.0,0,1.0,11
3,0.0,0,1.0,6
4,0.0,1,1.0,1


nssi_test frequencies by outcome


,si,nssi,att,count
0,0.0,0,0.0,608
1,1.0,1,0.0,23
2,0.0,1,0.0,18
3,1.0,1,1.0,14
4,0.0,1,1.0,1


Evaluating the models' ability to predict suicidal ideation

In [115]:
#Creating a function that predicts probabilities and evaluates the model
def eval(model, Xi_hold, y_hold, thresh):
    #Generating probability predictions
    proba=model.predict_proba(Xi_hold)
    proba=proba[:, 1]
    print('Probs: %.3f (%.3f)' % (mean(proba), std(proba)))
    #Evaluating probability predictions
    ypred=np.where(proba>thresh, 1, 0)
    f1= f1_score(y_hold, ypred)
    print(f'F1= {f1:f}')
    sens= recall(y_hold, ypred)
    print(f'Sensitivity= {sens:f}')
    tn, fp, fn, tp= confusion(y_hold, ypred).ravel()
    spec=tn/(tn+fp)
    print(f'Specificity= {spec:f}')
    auc= roc_auc_score(y_hold, proba)
    print(f'AUROC= {auc:f}')
    prec=ppv(y_hold, ypred)
    print(f'Precision= {prec:f}')

    return f1, sens, spec, auc, prec

In [116]:
#Creating dictionary of component and composite models
algos=['LR', 'RF', 'XGB']
threshs=[0.188868, 0.175579, 0.119263]
models1= {a: joblib.load(f'{a}_si.sav') for a in algos}
models2 = {a: joblib.load(f'{a}_sitbs.sav') for a in algos}


#Iterating through tuples to evaluate LR, RF, and XGB Models 
#Value of the tuple algo is paired to the other corresponding value in threshs, i.e. 'LR' is paired with threshold of 0.188868
def runevals(models1, Xi_hold, y_hold):
    for a, t in zip(algos, threshs):
        clf=models1[a]
        print(f'Metrics for {a} using threshold at {t}')
        eval(clf, Xi_hold, y_hold, t)
        print('\n')

In [117]:
runevals(models1, Xi_holds_si, y_holds_si)

Metrics for LR using threshold at 0.188868
Probs: 0.122 (0.106)
F1= 0.393162
Sensitivity= 0.479167
Specificity= 0.848684
AUROC= 0.746668
Precision= 0.333333


Metrics for RF using threshold at 0.175579
Probs: 0.139 (0.155)
F1= 0.475862
Sensitivity= 0.718750
Specificity= 0.794408
AUROC= 0.832811
Precision= 0.355670


Metrics for XGB using threshold at 0.119263
Probs: 0.122 (0.146)
F1= 0.444444
Sensitivity= 0.833333
Specificity= 0.697368
AUROC= 0.838739
Precision= 0.303030




In [118]:
runevals(models2, Xi_holds_si, y_holds_si)

Metrics for LR using threshold at 0.188868
Probs: 0.159 (0.122)
F1= 0.398714
Sensitivity= 0.645833
Specificity= 0.748355
AUROC= 0.741922
Precision= 0.288372


Metrics for RF using threshold at 0.175579
Probs: 0.173 (0.170)
F1= 0.449102
Sensitivity= 0.781250
Specificity= 0.731908
AUROC= 0.835364
Precision= 0.315126


Metrics for XGB using threshold at 0.119263
Probs: 0.157 (0.170)
F1= 0.447439
Sensitivity= 0.864583
Specificity= 0.684211
AUROC= 0.845498
Precision= 0.301818




Evaluating the models' ability to predict NSSI

In [127]:
threshs=[0.138621, 0.160344, 0.172887]
models1= {a: joblib.load(f'{a}_nssi.sav') for a in algos}

runevals(models1, Xi_holds_nssi, y_holds_nssi)

Metrics for LR using threshold at 0.138621
Probs: 0.071 (0.077)
F1= 0.349650
Sensitivity= 0.446429
Specificity= 0.898026
AUROC= 0.743348
Precision= 0.287356


Metrics for RF using threshold at 0.160344
Probs: 0.075 (0.091)
F1= 0.440476
Sensitivity= 0.660714
Specificity= 0.876645
AUROC= 0.851988
Precision= 0.330357


Metrics for XGB using threshold at 0.172887
Probs: 0.067 (0.096)
F1= 0.472441
Sensitivity= 0.535714
Specificity= 0.932566
AUROC= 0.848126
Precision= 0.422535




In [128]:
runevals(models2, Xi_holds_nssi, y_holds_nssi)

Metrics for LR using threshold at 0.138621
Probs: 0.154 (0.118)
F1= 0.259036
Sensitivity= 0.767857
Specificity= 0.616776
AUROC= 0.771734
Precision= 0.155797


Metrics for RF using threshold at 0.160344
Probs: 0.163 (0.163)
F1= 0.333333
Sensitivity= 0.821429
Specificity= 0.713816
AUROC= 0.852400
Precision= 0.209091


Metrics for XGB using threshold at 0.172887
Probs: 0.149 (0.165)
F1= 0.360656
Sensitivity= 0.785714
Specificity= 0.763158
AUROC= 0.856262
Precision= 0.234043




Evaluating the models' ability to predict attempts

In [121]:
threshs=[0.074037, 0.073374, 0.087420]
models1= {a: joblib.load(f'{a}_att.sav') for a in algos}

runevals(models1, Xi_holds_att, y_holds_att)

Metrics for LR using threshold at 0.074037
Probs: 0.042 (0.043)
F1= 0.223881
Sensitivity= 0.468750
Specificity= 0.856908
AUROC= 0.780530
Precision= 0.147059


Metrics for RF using threshold at 0.073374
Probs: 0.047 (0.074)
F1= 0.296774
Sensitivity= 0.718750
Specificity= 0.835526
AUROC= 0.881579
Precision= 0.186992


Metrics for XGB using threshold at 0.08742
Probs: 0.038 (0.062)
F1= 0.392157
Sensitivity= 0.625000
Specificity= 0.917763
AUROC= 0.883249
Precision= 0.285714




In [122]:
runevals(models2, Xi_holds_att, y_holds_att)

Metrics for LR using threshold at 0.074037
Probs: 0.149 (0.114)
F1= 0.122622
Sensitivity= 0.906250
Specificity= 0.322368
AUROC= 0.751234
Precision= 0.065760


Metrics for RF using threshold at 0.073374
Probs: 0.155 (0.152)
F1= 0.162025
Sensitivity= 1.000000
Specificity= 0.455592
AUROC= 0.873715
Precision= 0.088154


Metrics for XGB using threshold at 0.08742
Probs: 0.138 (0.152)
F1= 0.186667
Sensitivity= 0.875000
Specificity= 0.605263
AUROC= 0.854287
Precision= 0.104478




Predicting attempts with nssi model

In [129]:
y_hold_att=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\y_hold_att.csv')

threshs=[0.074037, 0.073374, 0.087420]
models1= {a: joblib.load(f'{a}_nssi.sav') for a in algos}

runevals(models1, Xi_hold, y_hold_att)

Metrics for LR using threshold at 0.074037
Probs: 0.075 (0.082)
F1= 0.150943
Sensitivity= 0.625000
Specificity= 0.694405
AUROC= 0.698126
Precision= 0.085837


Metrics for RF using threshold at 0.073374
Probs: 0.083 (0.099)
F1= 0.189781
Sensitivity= 0.812500
Specificity= 0.690100
AUROC= 0.838773
Precision= 0.107438


Metrics for XGB using threshold at 0.08742
Probs: 0.074 (0.104)
F1= 0.200000
Sensitivity= 0.593750
Specificity= 0.800574
AUROC= 0.823440
Precision= 0.120253




Deriving performance metrics for previously fitted models in full test dataset

In [130]:
#Loading datasets
y_hold_si=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\y_hold_si.csv')
y_hold_nssi=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\y_hold_nssi.csv')
y_hold_sitbs=pd.read_csv(r'C:\Users\z5291979\OneDrive - UNSW\Documents\lsac-data\processed_data\y_hold_sitbs.csv')

In [131]:
#SI Models
algos=['LR', 'RF', 'XGB']
threshs=[0.188868, 0.175579, 0.119263]
models= {a: joblib.load(f'{a}_si.sav') for a in algos}
runevals(models, Xi_hold, y_hold_si)

Metrics for LR using threshold at 0.188868
Probs: 0.123 (0.107)
F1= 0.380165
Sensitivity= 0.479167
Specificity= 0.842022
AUROC= 0.741830
Precision= 0.315068


Metrics for RF using threshold at 0.175579
Probs: 0.142 (0.158)
F1= 0.460000
Sensitivity= 0.718750
Specificity= 0.786730
AUROC= 0.825912
Precision= 0.338235


Metrics for XGB using threshold at 0.119263
Probs: 0.125 (0.152)
F1= 0.428954
Sensitivity= 0.833333
Specificity= 0.688784
AUROC= 0.831161
Precision= 0.288809




In [132]:
#NSSI Models
threshs=[0.138621, 0.160344, 0.172887]
models= {a: joblib.load(f'{a}_nssi.sav') for a in algos}
runevals(models, Xi_hold, y_hold_nssi)

Metrics for LR using threshold at 0.138621
Probs: 0.075 (0.082)
F1= 0.308642
Sensitivity= 0.446429
Specificity= 0.879643
AUROC= 0.727645
Precision= 0.235849


Metrics for RF using threshold at 0.160344
Probs: 0.083 (0.099)
F1= 0.375635
Sensitivity= 0.660714
Specificity= 0.845468
AUROC= 0.830198
Precision= 0.262411


Metrics for XGB using threshold at 0.172887
Probs: 0.074 (0.104)
F1= 0.413793
Sensitivity= 0.535714
Specificity= 0.912333
AUROC= 0.827332
Precision= 0.337079




In [133]:
#Attempt models
threshs=[0.074037, 0.073374, 0.087420]
models= {a: joblib.load(f'{a}_att.sav') for a in algos}

runevals(models, Xi_hold, y_hold_att)

Metrics for LR using threshold at 0.074037
Probs: 0.045 (0.048)
F1= 0.191083
Sensitivity= 0.468750
Specificity= 0.842181
AUROC= 0.766992
Precision= 0.120000


Metrics for RF using threshold at 0.073374
Probs: 0.054 (0.081)
F1= 0.231156
Sensitivity= 0.718750
Specificity= 0.793400
AUROC= 0.857223
Precision= 0.137725


Metrics for XGB using threshold at 0.08742
Probs: 0.045 (0.074)
F1= 0.312500
Sensitivity= 0.625000
Specificity= 0.890961
AUROC= 0.857335
Precision= 0.208333




In [134]:
#SITB Models
threshs=[0.208806, 0.152130, 0.176893]
models = {a: joblib.load(f'{a}_sitbs.sav') for a in algos}
runevals(models, Xi_hold, y_hold_sitbs)

Metrics for LR using threshold at 0.208806
Probs: 0.161 (0.125)
F1= 0.422360
Sensitivity= 0.561983
Specificity= 0.781250
AUROC= 0.710465
Precision= 0.338308


Metrics for RF using threshold at 0.15213
Probs: 0.176 (0.173)
F1= 0.474490
Sensitivity= 0.768595
Specificity= 0.707237
AUROC= 0.812860
Precision= 0.343173


Metrics for XGB using threshold at 0.176893
Probs: 0.161 (0.175)
F1= 0.498534
Sensitivity= 0.702479
Specificity= 0.777961
AUROC= 0.815959
Precision= 0.386364


